# Examining the effect of including glacial runoff in SPEI drought risk calculations

Welcome!  This notebook will reproduce and visualise the analyses behind Ultee, Coats & Mackay, "Glacial runoff buffers drought through the 21st century" (submitted to ESD), using helper functions from gSPEI.py.  All code is stored in a public GitHub repository--click [here](https://github.com/ehultee/glacial-SPEI) for access.

### Loading in modules and data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle
import gSPEI as gSPEI

In [ ]:
## Define the filepath - abbreviations indicate
## (P)arametric or (NP)nonparametric;
## Standardization (1) lumped or (2) split by starting month
fpath_NP2 = './data/SPEI_Files/nonparametric-var_stom_c/'

## Settings in filenames
integration_times = np.arange(3, 28, 4) # all SPEI integration times used
modelnames = ['CanESM2', 'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GISS-E2-R', 'INMCM4', 'MIROC-ESM', 'NorESM1-M'] # all models used in comparison
scenarios = ['Rcp4p5', 'Rcp8p5'] # climate scenarios
cases = ['NRunoff', 'WRunoff', 'diff'] # inclusion of glacier runoff

## Basins in the order they are written in each file
basin_names = ['INDUS','TARIM','BRAHMAPUTRA','ARAL SEA','COPPER','GANGES','YUKON','ALSEK','SUSITNA','BALKHASH','STIKINE','SANTA CRUZ',
'FRASER','BAKER','YANGTZE','SALWEEN','COLUMBIA','ISSYK-KUL','AMAZON','COLORADO','TAKU','MACKENZIE','NASS','THJORSA','JOEKULSA A F.',
'KUSKOKWIM','RHONE','SKEENA','OB','OELFUSA','MEKONG','DANUBE','NELSON RIVER','PO','KAMCHATKA','RHINE','GLOMA','HUANG HE','INDIGIRKA',
'LULE','RAPEL','SANTA','SKAGIT','KUBAN','TITICACA','NUSHAGAK','BIOBIO','IRRAWADDY','NEGRO','MAJES','CLUTHA','DAULE-VINCES',
'KALIXAELVEN','MAGDALENA','DRAMSELV','COLVILLE']

yrs = np.linspace(1900, 2101, num=2412) # time interval over which we have data

Now we will load data into a dictionary for one specific climate scenario, indexed by model name.  You can choose to analyse a different climate scenario or a different SPEI integration timescale by changing the arguments `itime` and `scen` below.

In [ ]:
itime = integration_times[0] # select timescale of integration. [0] is 3 months, used in manuscript.
scen = scenarios[0] # choose whether to load RCP 4.5 or RCP 8.5

SPEI_by_model = {m: {} for m in modelnames} # create dictionary indexed by model name
for m in modelnames:
    norunoff_f_m = fpath_NP2+'NRunoff_{}_{}_{}_Conduct.txt'.format(itime, m, scen)
    wrunoff_f_m = fpath_NP2+'WRunoff_{}_{}_{}_Conduct.txt'.format(itime, m, scen)
    SPEI_by_model[m]['NRunoff'] = np.loadtxt(norunoff_f_m)
    SPEI_by_model[m]['WRunoff'] = np.loadtxt(wrunoff_f_m)
    SPEI_by_model[m]['diff'] = SPEI_by_model[m]['WRunoff'] - SPEI_by_model[m]['NRunoff']

## Re-structure dictionary and create pandas DataFrames aggregated by basin
SPEI_by_basin_raw = gSPEI.sort_models_to_basins(SPEI_by_model)
SPEI_by_basin = {b: {} for b in basin_names}
for b in basin_names:
    for c in cases:
        SPEI_by_basin[b][c] = SPEI_by_basin_raw[b][c].fillna(-3) # fill negative excursions that were stored as NaNs

## Plot multi-GCM ensemble series for a given basin

First, we can plot time series of SPEI with and without glacial runoff included.  Examples of this kind of figure appear in Ultee, Coats & Mackay Figure 1.

We will use a helper function from gSPEI.py to compare running ensemble mean SPEI with no glacial runoff (orange) versus with glacial runoff (blue).  The function can also show SPEI time series for a single basin, separated by the global climate model that produced each series.  Choose the basin by its name from the `basin_names` list, above. 

By default, we plot a running mean so that long-term trends can be more easily seen.  If you prefer, you can adjust `window` to show time series with less smoothing (shorter rolling-mean windows).  You can also turn on or off the option to see single-GCM series along with the ensembles.

In [ ]:
example_b = 'COPPER' # name of basin to examine, selected from basin_names above
win = 30 # window in years over which to calculate the running mean
plot_single_models = True # choose whether to see individual GCMs plotted over ensembles

In [ ]:
## Compute multi-GCM ensemble means and quartiles
r_w = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'WRunoff').rolling(window=12*win).mean()
r_n = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'NRunoff').rolling(window=12*win).mean()
rm = SPEI_by_basin[example_b]['WRunoff'].rolling(window=12*30, axis=0).mean()
rm_q1 = rm.quantile(q=0.25, axis=1)
rm_q3 = rm.quantile(q=0.75, axis=1)
rm_n = SPEI_by_basin[example_b]['NRunoff'].rolling(window=12*30, axis=0).mean()
rm_q1_n = rm_n.quantile(q=0.25, axis=1)
rm_q3_n = rm_n.quantile(q=0.75, axis=1)

single_models_w = [SPEI_by_basin[example_b]['WRunoff'][m].rolling(window=12*win).mean() for m in modelnames]
single_models_n = [SPEI_by_basin[example_b]['NRunoff'][m].rolling(window=12*win).mean() for m in modelnames]

In [ ]:
colors_w = cm.get_cmap('Blues')(np.linspace(0.2, 1, num=len(modelnames)))
colors_n = cm.get_cmap('Wistia')(np.linspace(0.2, 1, num=len(modelnames)))
fig, ax = plt.subplots()
ax.plot(yrs, r_w, 'k', linewidth=3.0)
ax.plot(yrs, rm_q1, 'k')
ax.plot(yrs, rm_q3, 'k')
ax.plot(yrs, r_n, 'k', linewidth=3.0, ls=':')
ax.plot(yrs, rm_q1_n, 'k', ls=':')
ax.plot(yrs, rm_q3_n, 'k', ls=':')
if plot_single_models:
    for i in range(len(modelnames)):
        ax.plot(yrs, single_models_w[i], color=colors_w[i], alpha=0.5)
        ax.plot(yrs, single_models_n[i], color=colors_n[i], alpha=0.5)
ax.fill_between(yrs, rm_q1, rm_q3, color='DarkBlue', alpha=0.2)
ax.fill_between(yrs, rm_q1_n, rm_q3_n, color='DarkOrange', alpha=0.2)
ax.tick_params(axis='both', labelsize=12)
ax.set_xticks([2000, 2050, 2100])
ax.set_xlim(1980, 2100)
ax.set_xlabel('Years', fontsize=14)
ax.set_ylabel('Rolling mean SPEI', fontsize=14)
plt.tight_layout()

## Compare multi-GCM ensemble series across basins

Figure 1 of Ultee, Coats & Mackay shows four example basins with different projected effects.  We reproduce that figure below.

In [ ]:
example_basins = ('COPPER', 'TARIM', 'RHONE', 'MAJES')

color_fam = cm.get_cmap('tab20b')
color_with = color_fam(0)
color_no = color_fam(10)
fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, 
                                              tight_layout=True, figsize=(9,6))
for example_b, ax in zip(example_basins, (ax1,ax2,ax3,ax4)):
    r_w = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'WRunoff').rolling(window=12*30).mean()
    r_n = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'NRunoff').rolling(window=12*30).mean()
    rm = SPEI_by_basin[example_b]['WRunoff'].rolling(window=12*30, axis=0).mean()
    rm_q1 = rm.quantile(q=0.25, axis=1, interpolation='lower')
    rm_q3 = rm.quantile(q=0.75, axis=1, interpolation='higher')
    rm_n = SPEI_by_basin[example_b]['NRunoff'].rolling(window=12*30, axis=0).mean()
    rm_q1_n = rm_n.quantile(q=0.25, axis=1, interpolation='lower')
    rm_q3_n = rm_n.quantile(q=0.75, axis=1, interpolation='higher')
    
    ax.plot(yrs, r_w, 'k', linewidth=3.0)
    ax.plot(yrs, rm_q1, 'k')
    ax.plot(yrs, rm_q3, 'k')
    ax.plot(yrs, r_n, 'k', linewidth=3.0, ls=':')
    ax.plot(yrs, rm_q1_n, 'k', ls=':')
    ax.plot(yrs, rm_q3_n, 'k', ls=':')
    # for i in range(len(modelnames)):
    #     ax.plot(yrs, single_models_w[i], color=colors_w[i], alpha=0.5)
    #     ax.plot(yrs, single_models_n[i], color=colors_n[i], alpha=0.5)
    ax.fill_between(yrs, rm_q1, rm_q3, color=color_with, alpha=0.4)
    ax.fill_between(yrs, rm_q1_n, rm_q3_n, color=color_no, alpha=0.4)
    ax.tick_params(axis='both', labelsize=12)
    ax.set_xticks([2000, 2050, 2100])
    ax.set_xlim(1980, 2100)
    extra = Rectangle((0,0), 0.1, 0.1, fc='w', fill=False, 
                      edgecolor='none', linewidth=0) # invisible marker for a legend entry
    leg = ax.legend([extra], [example_b], loc='best', 
                    handlelength=0, handletextpad=0, fancybox=True,
                    prop={'size':13})
for ax in (ax3, ax4):
    ax.set_xlabel('Year', fontsize=14)
for ax in (ax1, ax3):
    ax.set_ylabel('Rolling mean SPEI', fontsize=14)
fig1.align_ylabels()

## Multi-basin summary: change in SPEI at end of century

The functions above show timeseries and running statistics for individual basins.  But what can be said in general?  When we account for glacial runoff in our SPEI calculation, how does the mean SPEI change?  Does SPEI become more or less variable?

We calculate the mean SPEI with and without glacial runoff, for each of our 8 climate models, in each of our 56 basins, over a 30-year period.  We do the same for variance.  Then, we plot markers with whiskers to visualize the inter-model range in each.  

Figure 2 of Ultee, Coats & Mackay examines these summary results for the 30-year period at the end of the 21st century, but you can explore other periods by adjusting `timepd` below.  For simplicity, we show a single scenario here.  If you'd like to examine a different climate scenario, you can return to the top of the notebook and change the climate scenario setting `scen`.  The code to reproduce Figure 2 directly is also included in this repo, as `mean_var_shift-perscenario.py`.

In [ ]:
## Calculate changes due to glacial effect at end of century, using gSPEI functions
timepd = (2070, 2100)

## Calculate changes due to glacial effect at end of century, using ensemble approach
meandiff, quantile_spread = gSPEI.ensemble_glacial_meandiff(SPEI_by_basin, years=timepd)
vardiff, var_spread = gSPEI.ensemble_glacial_vardiff(SPEI_by_basin, years=timepd)

## Calculate full multi-GCM range, for comparison
_, mean_spread_full = gSPEI.glacial_meandiff(SPEI_by_model, years=timepd)
_, var_spread_full = gSPEI.glacial_vardiff(SPEI_by_model, years=timepd)

fig1, ax1 = plt.subplots(figsize=(5,4))
ax1.axhline(y=0, ls=':', color='Grey', alpha=0.5)
ax1.axvline(x=0, ls=':', color='Grey', alpha=0.5)
ax1.errorbar(x=meandiff, y=vardiff, xerr=quantile_spread, yerr=var_spread, 
             ls='', marker='d', elinewidth=2.0, color='k')
ax1.errorbar(x=meandiff, y=vardiff, xerr=mean_spread_full, yerr=var_spread_full, 
             ls='', marker='d', elinewidth=1.0, color='k', alpha=0.5) #extend whiskers to full range
ax1.set_xlabel('Difference in mean SPEI', fontsize=16)
ax1.set_ylabel('Difference in SPEI variance', fontsize=16)
ax1.set(ylim=(-1, 2), yticks=(-1, 0, 1, 2),
        xlim=(-0.1, 2), xticks=(0,1,2))
ax1.tick_params(axis='both', labelsize=12)
plt.tight_layout()
plt.show()

## Bonus: SPEI shifts over time
SPEI responds to changes in global and regional climate over time.  So, it may be of interest to compare 30-year means from a period in the 20th century versus a period later in the 21st century.  This preliminary analysis was not included in the Ultee & Coats manuscript.

We compute single-model mean and variance of SPEI for each basin, for two 30-year periods.  Below, we choose the historical period 1950-1980, before glacial runoff from the Huss & Hock model is introduced (1980), and the future period 2070-2100, at the end of the 21st century.  The plot is in the same style as the one above.  Remember, this plot shows shift in SPEI statistics over time rather than the shift in a single time period due to glacial effects.

In [ ]:
## 30-yr period means and variance
modelmeans_1950_1980 = {m: [] for m in modelnames} #dictionary indexed by model name
modelvar_1950_1980 = {m: [] for m in modelnames}
modelmeans_2070_2100 = {m: [] for m in modelnames}
modelvar_2070_2100 = {m: [] for m in modelnames}
mean_shifts = {m: [] for m in modelnames}
var_shifts = {m: [] for m in modelnames}

for m in modelnames:
    means_i = [np.nanmean(SPEI_by_model[m]['diff'][j][600:971]) for j in range(len(basin_names))]
    var_i = [np.nanvar(SPEI_by_model[m]['diff'][j][600:971]) for j in range(len(basin_names))]
    means_f = [np.nanmean(SPEI_by_model[m]['diff'][j][2039:2410]) for j in range(len(basin_names))]
    var_f = [np.nanvar(SPEI_by_model[m]['diff'][j][2039:2410]) for j in range(len(basin_names))]
    modelmeans_1950_1980[m] = means_i
    modelvar_1950_1980[m] = var_i
    modelmeans_2070_2100[m] = means_f
    modelvar_2070_2100[m] = var_f
    mean_shifts[m] = np.array(means_f) - np.array(means_i)
    var_shifts[m] = np.array(var_f) - np.array(var_i)

basin_mean_shifts = {b: [] for b in basin_names} #dictionary indexed by basin name
basin_var_shifts = {b: [] for b in basin_names} #dictionary indexed by model name
basin_meanshift_meds = [] #arrays for plotting
basin_varshift_meds = []
basin_meanshift_range = []
basin_varshift_range = []

for i, b in enumerate(basin_names):
    bmeans_i = [np.nanmean(SPEI_by_model[m]['WRunoff'][i][600:971]) for m in modelnames]
    bvar_i = [np.nanvar(SPEI_by_model[m]['WRunoff'][i][600:971]) for m in modelnames]
    bmeans_f = [np.nanmean(SPEI_by_model[m]['WRunoff'][i][2039:2410]) for m in modelnames]
    bvar_f = [np.nanvar(SPEI_by_model[m]['WRunoff'][i][2039:2410]) for m in modelnames]
    basin_mean_shifts[b] = np.array(bmeans_f) - np.array(bmeans_i)
    basin_var_shifts[b] = np.array(bvar_f) - np.array(bvar_i)
    basin_meanshift_meds.append(np.nanmedian(basin_mean_shifts[b]))
    basin_varshift_meds.append(np.nanmedian(basin_var_shifts[b]))
    basin_meanshift_range.append(np.nanmax(basin_mean_shifts[b]) - np.nanmin(basin_mean_shifts[b]))
    basin_varshift_range.append(np.nanmax(basin_var_shifts[b]) - np.nanmin(basin_var_shifts[b]))


plt.figure('Mean and variance shifts, 2070-2100 versus 1950-1980, per basin for WRunoff case')
plt.errorbar(x=basin_meanshift_meds, y=basin_varshift_meds, xerr=basin_meanshift_range, yerr=basin_varshift_range, ls='')
plt.axes().set_xlabel('Difference in 30-yr mean SPEI', fontsize=16)
plt.axes().set_ylabel('Difference in SPEI variance', fontsize=16)
plt.axes().set_ylim(-0.5, 2.5)
plt.axes().set_xlim(-1, 5)
plt.show()